<a href="https://colab.research.google.com/github/Akhil-blanc/Implementation-FaceNet-SiameseNet/blob/main/Copy_of_siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim 
import torch.nn.functional as F

In [2]:
train_data = datasets.LFWPairs(root="/content",
                              split='train',
                              image_set='deepfunneled',
                              transform =transforms.Compose([transforms.Resize((220, 220)), transforms.PILToTensor(), transforms.ConvertImageDtype(dtype = torch.float32)]),
                              target_transform=None,
                              download=True)

  0%|          | 0/108761145 [00:00<?, ?it/s]

Extracting /content/lfw-py/lfw-deepfunneled.tgz to /content/lfw-py


  0%|          | 0/56579 [00:00<?, ?it/s]

In [3]:
test_data = datasets.LFWPairs(root="/content",
                              split='test',
                              image_set='deepfunneled',
                              transform =transforms.Compose([transforms.Resize((220, 220)), transforms.PILToTensor(), transforms.ConvertImageDtype(dtype = torch.float32)]),
                              target_transform=None,
                              download=True)


Using downloaded and verified file: /content/lfw-py/lfw-deepfunneled.tgz
Extracting /content/lfw-py/lfw-deepfunneled.tgz to /content/lfw-py


  0%|          | 0/26002 [00:00<?, ?it/s]

In [4]:
#create the Siamese Neural Network
class SiameseNetwork(nn.Module):

    def __init__(self):
        super(SiameseNetwork, self).__init__()

        #Setting up the Sequential of the CNN layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size = 10, stride = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride = 2),

            nn.Conv2d(64, 128, kernel_size = 7, stride = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride = 2),

            nn.Conv2d(128, 128, kernel_size = 4, stride = 1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride = 2),

            nn.Conv2d(128, 256, kernel_size = 4, stride = 1),
            nn.ReLU(inplace=True),

            nn.Flatten(1, -1)
        )
        #Setting up the Fully Connected Layers
        self.fc1 = nn.Sequential(
            nn.Linear(9216, 4096),
            nn.Sigmoid()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(4096, 1),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        #This function will be declared for both the images
        #It's output is used to determine the similarity
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        #Here in this function we pass in both the images and obtain the corresponding vectors that are returned.
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        L1_dist = torch.sub(output1, output2)

        L1_dist = torch.abs(L1_dist)
        
        output = self.fc2(L1_dist)

        return output
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, mean=0.0, std=0.01)
                if m.bias is not None:
                    nn.init.normal_(m.bias, mean=0.5, std=0.01)

            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0.0, std=0.2)
                nn.init.normal_(m.bias, mean=0.5, std=0.01)
    

In [5]:
class BCEloss(torch.nn.Module):
    def __init__(self):
        super(BCEloss, self).__init__()

    def forward(self, output, label):
        #Calculate the Binary Crossentropy Loss
        loss_BCE = torch.mean(label*output.log() + (1-label)*(1-output).log())
        print(loss_BCE.shape)
        return (-1)*loss_BCE
# from torchvision import models
# from torchsummary import summary
# summary(BCEloss,[(1,), (1,)])

In [6]:
model = SiameseNetwork()
loss = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0005)
i1 = torch.randn(1,105,105)
i2 = torch.randn(1,105,105)
i1 = torch.reshape(i1,(1,1,105,105))
i2 = torch.reshape(i2,(1,1,105,105))
out = model(i1,i2)

In [7]:
from torchvision import models
from torchsummary import summary

summary(model,[(1,105,105), (1,105,105)])

RuntimeError: ignored

In [23]:
def train(model):
    # default_transform = transforms.Compose([
    #     transforms.Scale(128),
    #     transforms.ToTensor(),
    # ])
    train_dataset = datasets.LFWPairs(root="/content",
                              split='train',
                              image_set='deepfunneled',
                              transform =transforms.Compose([transforms.Resize((105, 105)),
                                                             transforms.Grayscale(),
                                                             transforms.PILToTensor(),
                                                             transforms.ConvertImageDtype(dtype = torch.float32)]),
                              target_transform=None,
                              download=True)
    print("Loaded {} training data.".format(len(train_dataset)))
    
    # # Data Loader (Input Pipeline)
    criterion = nn.BCELoss()
    siamese_net = SiameseNetwork()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device=='cuda':
        criterion = criterion.cuda()
        siamese_net = siamese_net.cuda()
    kwargs = {'num_workers': 4, 'pin_memory': True} if device=='cuda' else {}
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=32,
                                               shuffle=False,
                                               **kwargs)
    # # Loss and Optimizer

    optimizer = torch.optim.SGD(siamese_net.parameters(), lr=0.001, momentum = 0.9)
    lambda1 = lambda epoch: 0.99
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
    # Train the Model
    num_epochs = 10
    
    for epoch in range(num_epochs):
        for i, (img1_set, img2_set, labels) in enumerate(train_loader):

            if device=='cuda':
                img1_set = img1_set.cuda()
                img2_set = img2_set.cuda()
                labels = labels.cuda()

            img1_set = Variable(img1_set)
            img2_set = Variable(img2_set)
            labels = Variable(labels.view(-1, 1).float())

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            # if args.contra_loss:
            #     output1, output2 = siamese_net(img1_set, img2_set)
            #     loss = criterion(output1, output2, labels)
            #     loss.backward()
            #     optimizer.step()
            # else:
            output_labels_prob = siamese_net(img1_set, img2_set)
            loss = criterion(output_labels_prob, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//128, loss.item()))
        scheduler.step()
    # Training accuracy
    # test_against_data(args, 'training', train_loader, siamese_net)

    return siamese_net


In [24]:
train(model)

Files already downloaded and verified
Loaded 2200 training data.
Epoch [1/10], Iter [1/17] Loss: 0.6944
Epoch [1/10], Iter [2/17] Loss: 0.6942
Epoch [1/10], Iter [3/17] Loss: 0.6937
Epoch [1/10], Iter [4/17] Loss: 0.6930
Epoch [1/10], Iter [5/17] Loss: 0.6922
Epoch [1/10], Iter [6/17] Loss: 0.6911
Epoch [1/10], Iter [7/17] Loss: 0.6900
Epoch [1/10], Iter [8/17] Loss: 0.6886
Epoch [1/10], Iter [9/17] Loss: 0.6873
Epoch [1/10], Iter [10/17] Loss: 0.6858
Epoch [1/10], Iter [11/17] Loss: 0.6842
Epoch [1/10], Iter [12/17] Loss: 0.6824
Epoch [1/10], Iter [13/17] Loss: 0.6808
Epoch [1/10], Iter [14/17] Loss: 0.6789
Epoch [1/10], Iter [15/17] Loss: 0.6771
Epoch [1/10], Iter [16/17] Loss: 0.6752
Epoch [1/10], Iter [17/17] Loss: 0.6732
Epoch [1/10], Iter [18/17] Loss: 0.6712
Epoch [1/10], Iter [19/17] Loss: 0.6692
Epoch [1/10], Iter [20/17] Loss: 0.6671
Epoch [1/10], Iter [21/17] Loss: 0.6650
Epoch [1/10], Iter [22/17] Loss: 0.6629
Epoch [1/10], Iter [23/17] Loss: 0.6607
Epoch [1/10], Iter [24/1

KeyboardInterrupt: ignored

In [ ]:
vgg16 = torchvision.models.vgg16_bn(weights = 'IMAGENET1K_V1')
print(vgg16.classifier[6].out_features) # 1000 


# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False

# Newly created modules have require_grad=True by default
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-4] # Remove last layer
features.extend([nn.Linear(num_features, 4096), nn.ReLU(inplace = True), nn.Linear(4096,4096)]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
print(vgg16)
summary(vgg16,(3,250,250))

1000
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128

In [ ]:
class SiameseNetwork2(nn.Module):

    def __init__(self):
        super(SiameseNetwork2, self).__init__()

        #Setting up the Sequential of the CNN layers
        self.vgg = torchvision.models.vgg16_bn(weights = 'IMAGENET1K_V1')
        for param in self.vgg.features.parameters():
            param.require_grad = False

        # Newly created modules have require_grad=True by default
        num_features = self.vgg.classifier[6].in_features
        features = list(self.vgg.classifier.children())[:-4] # Remove last layer
        features.extend([nn.Linear(num_features, 4096), nn.ReLU(inplace = True), nn.Linear(4096,4096)]) # Add our layer with 4 outputs
        self.vgg.classifier = nn.Sequential(*features) # Replace the model classifier
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 1),
            nn.Sigmoid()
        )
        
    def forward_once(self, x):
        #This function will be declared for both the images
        #It's output is used to determine the similarity
        output = self.vgg(x)
        return output

    def forward(self, input1, input2):
        #Here in this function we pass in both the images and obtain the corresponding vectors that are returned.
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        L1_dist = torch.sub(output1, output2)

        L1_dist = torch.abs(L1_dist)
        
        output = self.fc2(L1_dist)

        return output

In [ ]:
def train2():
    # default_transform = transforms.Compose([
    #     transforms.Scale(128),
    #     transforms.ToTensor(),
    # ])
    train_dataset = datasets.LFWPairs(root="/content",
                              split='train',
                              image_set='deepfunneled',
                              transform =transforms.Compose([transforms.PILToTensor(),
                                                             transforms.ConvertImageDtype(dtype = torch.float32)]),
                              target_transform=None,
                              download=True)
    print("Loaded {} training data.".format(len(train_dataset)))

    # # Data Loader (Input Pipeline)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=32,
                                               shuffle=True)

    siamese_net = SiameseNetwork2()
    # if args.cuda:
    #     siamese_net = siamese_net.cuda()

    # # Loss and Optimizer
    criterion = nn.BCELoss()

    optimizer = torch.optim.Adam(siamese_net.parameters(), lr=0.001)

    # Train the Model
    num_epochs = 1
    for epoch in range(num_epochs):
        for i, (img1_set, img2_set, labels) in enumerate(train_loader):

            # if args.cuda:
            #     img1_set = img1_set.cuda()
            #     img2_set = img2_set.cuda()
            #     labels = labels.cuda()

            img1_set = Variable(img1_set)
            img2_set = Variable(img2_set)
            labels = Variable(labels.view(-1, 1).float())

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            # if args.contra_loss:
            #     output1, output2 = siamese_net(img1_set, img2_set)
            #     loss = criterion(output1, output2, labels)
            #     loss.backward()
            #     optimizer.step()
            # else:
            output_labels_prob = siamese_net(img1_set, img2_set)
            loss = criterion(output_labels_prob, labels)
            loss.backward()
            optimizer.step()
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//128, loss.item()))

    # Training accuracy
    # test_against_data(args, 'training', train_loader, siamese_net)

    return siamese_net


In [ ]:
train2()

Files already downloaded and verified
Loaded 2200 training data.
